In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

**Importando e apresentando a base de dados pokemon**

In [ ]:
df = pd.read_csv("../input/pokemon.csv")

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.sample(10) # Pegando 10 registros aleatórios na base

**Avaliando as informações presentes no dataframe**

In [ ]:
df.info()

Observa-se que o shape da tabela é de 800 linhas por 12 colunas. E dentre essas colunas, tem-se 3 variáveis do tipo obj, 8 variáveis do tipo inteiro e uma booleana.

Além disso, observa-se que há número valores com NaN preenchidos. Há 1 registro em "Name" e 386 registros em Type 2. Vamos iniciar o tratamento nesses valores.

In [ ]:
df.shape

![](http://)**Tratamento dos dados NaN**

In [ ]:
# Descobrindo qual o valor NaN no campo "Name"
df[df['Name'].isna() == True]

Observa-se que há valores preenchidos para este pokemon, embora ele não tenha nome. Vamos excluir esse registro, uma vez que não podemos identificar a qual pokemon esses atributos se referem.

In [ ]:
df.dropna(subset=['Name'], inplace=True)
df.info()

Pronto! Agora observa-se que a linha sem nome foi deletada. Vamos observar se é necessário resetar os IDs dos pokemons.

In [ ]:
df.loc[60:70] # Apresentandos os registros de 59 a 69

Observamos que o registro 63 não existe mais. Logo, vamos resetar nossa contagem de index e excluir as colunas index e #, uma vez que agora elas armazenam informações irrelevantes para nós. Iremos utilizar apenas o ID gerado pelo dataframe, que se inicia com 0 até o valor 798.

In [ ]:
df.reset_index(inplace=True)
df.loc[60:70]

In [ ]:
df.drop(['index', '#'], axis=1, inplace=True) # Removendo as duas colunas index e #
df.head()

O outro campo que possuia valores NaN era o "Type 2", no entanto, de acordo com as informações do dataset, os NaN presentes neste campo sinalizam que o pokemon em questão tem apenas um tipo, ao inves de dois. Logo, não podemos excluir ou desconsiderar essa informação. Além disso, vamos manter os valores como NaN mesmo. Isso não derá impacto em nossa análise descritiva.

Feito a limpeza de dados, vamos iniciar a análise exploratória dos dados.

**Análise Exploratória dos dados**

Vamos iniciar a análise observando as distribuições dos dados de batalha das variáveis categóricas (HP, Attack, Defense, Sp. Atk, Sp.Def, Speed e Generation).


In [ ]:
df.plot(title='HP', kind='hist', y='HP',
               alpha=0.5, color='red')

df.plot(title='Attack', kind='hist', y='Attack',
               alpha=0.5, color='green')

df.plot(title='Defense', kind='hist', y='Defense',
               alpha=0.5, color='blue')

df.plot(title='Sp. Atk', kind='hist', y='Sp. Atk',
               alpha=0.5, color='black')

df.plot(title='Sp. Def', kind='hist', y='Sp. Def',
               alpha=0.5, color='yellow')

df.plot(title='Speed', kind='hist', y='Speed',
               alpha=0.5, color='orange')

df.plot(title='Generation', kind='hist', y='Generation',
               alpha=0.5, color='orange')

plt.show()

Observa-se que boa parte das distribuições se aproximam de uma distribuição normal, com exceção da variável Generation. Agora, vamos avaliar brevemente algumas estatisticas base do nosso dataframe.

In [ ]:
df.describe()

Outra informação estatisticamente importante, é a matriz de correlação.

In [ ]:
_,ax = plt.subplots(figsize=(10, 10))
sns.heatmap(df.corr(), annot=True, linewidths=.2, fmt= '.1f',ax=ax)
plt.show()

Agora vamos avaliar os atributos categóricos da base.

In [ ]:
plt.subplots(figsize=(20, 5))
sns.countplot(x='Type 1', data=df)
plt.show()

In [ ]:
plt.subplots(figsize=(20, 5))
sns.countplot(x='Type 2', data=df)
plt.show()

Notamos que em relação ao Type 1, ou seja, o tipo base do pokemon, os tipos mais comuns são "Water" e "Normal". E "Fly" e "Fairy" são os tipos com menos pokemons.
Já em relação ao Type 2, o tipo "Flying" é disparado o mais comum, e o tipo "Bug" o menos.

Vamos avaliar os atributos bases de batalha, especificamente HP, Attack e Defense de acordo com cada tipo.

In [ ]:
plt.subplots(figsize=(20, 5))
sns.boxplot(x='Type 1', y='HP', data=df)
plt.show()

O tipo Dragon tem, de modo geral, HP maiores que os outros tipos. Além disso, observam-se alguns outliers, principalmente no tipo Normal. Para este caso pode-se abstratir três hipoteses: de que esses outliers são pokemons lendarios; ou podem ter sido preenchidos erroneamente na base de dados; ou que realmente são pokemons com um HP muito maior que a maioria dos outros pokemons.

In [ ]:
plt.subplots(figsize=(20, 5))
sns.boxplot(x='Type 1', y='Attack', data=df)
plt.show()

Aqui também observamos que o tipo Dragon tem, em média, o maior valor de Attack, e os Fairy os menores.

In [ ]:
plt.subplots(figsize=(20, 5))
sns.boxplot(x='Type 1', y='Defense', data=df)
plt.show()

Já com relação a defesa, os pokemons tipo Steel tem, em média, uma defesa muito maior que as dos outros pokemons.

Vamos avaliar se os valores outliers no atributo HP são pokemons lendários ou se são erros na base.

In [ ]:
sns.factorplot(x='Type 1', y='HP', data=df, col='Legendary', kind='box')

Acabamos de descobrir com base nos gráficos acima, apesar de estarem de ruim de ver, mas para essa questão são suficientes para interpretar que os valores outliers no tipo Normal não são pokemons lendários. Logo, eles podem ser ou erro na base de dados ou realmente podemos com HP muito alto, foram do comum.  Vamos descobrir quem são esses pokemons.

In [ ]:
df_normal = df[df['Type 1'] == 'Normal']
df_normal.nlargest(20, 'HP')

Aqui descobrimos que os dois valores outliers, do tipo normal, em relação ao atributo HP são os pokemons: Blissey e o Chansey.

Vamos agora avaliar os pokemons lendários agora.

In [ ]:
df['Legendary'].value_counts() # Quantidade de pokemons lendários

In [ ]:
# Imprimindo o percentual de lendários em um gráfico de pizza
plt.pie(df['Legendary'].value_counts(), labels=("Normal", "Lendário"), 
        autopct='%1.1f%%', shadow=True, startangle=90)

In [ ]:
df_lendarios = df[df['Legendary']]
df_lendarios

In [ ]:
plt.subplots(figsize=(20, 5))
sns.violinplot(x='Type 1', y='HP', data=df)
plt.show()

In [ ]:
sns.factorplot(x='Generation', y='HP', data=df, col='Type 1', kind='violin')

Aqui conseguimos analisar como é a relação de lendários de acordo com cada geral. Notamos brevemente que as primeiras gerações (1 e 2) do tipo normal, possuem uma média HP bem superior do que em comparação com outras gerações de outros tipos também.

Vamos agora avaliar que pokemons são mais fortes, para isso vamos criar dois novos atributos:
* um índice de equilibrio que corresponde a diferença entre o Attack e a Defense (ou seja, quanto mais próximo de 0, mais equilibrado para batalha o pokemon é, quanto maior significa que esse pokemons é voltado principalmente para atacar e quanto menor significa que ele é voltado principalmente para defender)
* e um índice de poder total que correponde ao somatório do Attack, Defense e HP (quanto maior esse indice, melhor é esse pokemon)

In [ ]:
df['indice_equilibrio'] = df['Attack'] - df['Defense']
df['indice_poder'] = df['HP'] + df['Attack'] + df['Defense']
df.head()

Vamos ver agora quais são os pokemons: mais fortes, mais fracos, os melhores para atacar e os melhores para defender.

In [ ]:
df.plot(title='indice_equilibrio', kind='hist', y='indice_equilibrio',
               alpha=0.5, color='red')

df.plot(title='indice_poder', kind='hist', y='indice_poder',
               alpha=0.5, color='green')

Conforme a distribuição do índice de equilibrio, observamos que a média é que os pokemons são mais equilibrado do que focados no ataque ou defesa.

E que, conforme a distribuição do índice de poder, os pokemons tem um poder médio entre 150 e 300.

In [ ]:
df.nlargest(20, 'indice_equilibrio') # Os 20 Pokemons mais fortes para atacar

In [ ]:
df.nsmallest(20, 'indice_equilibrio') # Os 20 Pokemons mais fortes para defender

In [ ]:
df_mais_fortes = df.nlargest(20, 'indice_poder') # Os 20 Pokemons mais fortes

# Gerando o plot dos 20 Pokemons mais fortes
plt.subplots(figsize=(10,5))
plt.grid(True, linestyle='--')
plt.title('Pokemons mais fortes')
plt.plot(df_mais_fortes['Name'], df_mais_fortes['indice_poder'], label='Índice poder', marker='o')
plt.xticks(rotation=90)
plt.xlabel('Nome do Pokemon')
plt.ylabel('Ìndice de Poder')
plt.legend()
plt.show()

In [ ]:
df_mais_fracos = df.nsmallest(20, 'indice_poder') # Os 20 Pokemons mais fracos

# Gerando o plot dos 20 Pokemons mais fracos
plt.subplots(figsize=(10,5))
plt.grid(True, linestyle='--')
plt.title('Pokemons mais fracos')
plt.plot(df_mais_fracos['Name'], df_mais_fracos['indice_poder'], label='Índice poder', marker='o')
plt.xticks(rotation=90)
plt.xlabel('Nome do Pokemon')
plt.ylabel('Índice de Poder')
plt.legend()
plt.show()

In [ ]:
df.describe() # Avaliando as estatisticas básicas dos dois novos campos

In [ ]:
df.plot(kind = "scatter",x="indice_poder",y = "indice_equilibrio")

Com base no scatter plot acima, observa-se que, de modo geral, há um equilíbrio entre os pokemons de todas as gerações. Ou seja, os pokemos muito bons para defesa ou ataca, tem um índice de poder na média (aproximadamente 222~250). Embora, pokemons com índice de poder alto, que são possivelmente lendários, tendem a ser menos equilibrados (eles estão localizados no terceiro quartil para cima) diferentemente dos pokemons da mediana para baixo, que são bem mais equilibrados. 